## "Pop-up Housing" Business Project - Identifying Neighboods/Businesses
-- Final Project for Class: Capstone Project - The Battle of Neighborhoods <br> Week 5 Project - Analysis for opening a business

The Sections included in thie workbook<BR>
    Sect 1 - Set-up: Loading Pages<BR>
    Sect 2 - Geo data - Get Zip codes for Toronto area<BR>
    Sect 3 - Import Business Directory data from FourSquare<BR>
    Sect 4 - Compare venue category counts in each Neighborhood for ALL Venue Categories<BR>
    Sect 5 - Compare venue category counts in each Neighborhood for TARGETED Venue Categories<BR>
    <hr>

## Sect 1 - Set-up: Loading Pages<BR>

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#import requests # library to handle requests
#import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation



In [ ]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

print('geopy')

Solving environment: - 

In [143]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
print('ipython')

ipython


In [144]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print('pandas')

pandas


In [145]:
!python3 -m pip install folium

In [3]:
import folium

ModuleNotFoundError: No module named 'folium'

In [147]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [148]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [149]:
# verify packages are installed -- folium?
# !pip list --isolated

## Sect 2 - Geo data - Get Zip codes for Toronto area

In [150]:
# download wikipedia 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
webcopy = requests.get(url).text
postaldata = BeautifulSoup(webcopy, 'lxml')

In [151]:
# create the new Toronto Dataframe
columnlabel = ['Postalcode','Borough','Neighborhood']
tor_codes = pd.DataFrame(columns = columnlabel)

In [152]:
tor_codes.head()

,Postalcode,Borough,Neighborhood


In [153]:
# read through table to get postcode, borough, neighborhood 
wikicontent = postaldata.find('div', class_='mw-parser-output')
table = wikicontent.table.tbody
postcode = 0
borough = 0
neighborhood = 0

In [154]:
tor_codes.head()

,Postalcode,Borough,Neighborhood


In [155]:
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n').replace(']','')  # postcode = td.text
            i = i + 1
            # print(postcode,"tim1",i)  #added
        elif i == 1:
            borough = td.text.strip('\n').replace(']','')   # borough1 = td.text
            i = i + 1
            # print(borough,i)  #added
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
            # print(neighborhood,"tim3",i)  #added
    tor_codes = tor_codes.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [156]:
tor_codes.head()

,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [157]:
# clean dataframe 
tor_codes = tor_codes[tor_codes.Borough!='Not assigned']
tor_codes = tor_codes[tor_codes.Borough!= 0]
tor_codes.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,tor_codes.shape[0]):
    if tor_codes.iloc[i][2] == 'Not assigned':
        tor_codes.iloc[i][2] = tor_codes.iloc[i][1]
        i = i+1

In [158]:
tor_codes.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [159]:
df = tor_codes.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
# drop burroughs with none assigned
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [160]:

tor_codes.head(20)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [161]:
print(tor_codes.shape)

(103, 3)


### 2a) Get Long/Lat

In [162]:
# Read the cvs file and convert it to a dataframe

url='http://cocl.us/Geospatial_data'
df_pcodesll=pd.read_csv(url)
df_pcodesll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [163]:
#Change "Postal Code" to "Postalcode" for proper merger
df_pcodesll.columns = ['Postalcode', 'Latitude', 'Longitude']
df_pcodesll.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [164]:
nhs=pd.merge(tor_codes,df_pcodesll, how='right', on = 'Postalcode')
nhs.head(111)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### 2b) Explore and cluster the neighborhoods in Toronto. 


Part A) I worked with the Borough that contained the word Toronto and then used CircleMap flags to identify each on a map in different colors.

Part B) I accessed FourSquare to find places near the Neighborhood that has a "Airport".  The goal is to fly to Toronto and have a pubcrawl and then walk back to the airport. So I looked for places within 800 meters.

Hints from Course Admin
- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visalize your neighborhoods and how they cluster together.

--- Group neighborhoods with like burroughs

In [165]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))

Full Toronto -- Postal Code DF

In [166]:
grp = nhs.groupby([ 'Borough','Postalcode', 'Latitude','Longitude'])
tor_neigh = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
tor_neigh.head(100)

,Borough,Postalcode,Latitude,Longitude,Neighborhood
0,Central Toronto,M4N,43.728020,-79.388790,Lawrence Park
1,Central Toronto,M4P,43.712751,-79.390197,Davisville North
2,Central Toronto,M4R,43.715383,-79.405678,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,43.704324,-79.388790,Davisville
4,Central Toronto,M4T,43.689574,-79.383160,"Moore Park, Summerhill East"
5,Central Toronto,M4V,43.686412,-79.400049,"Summerhill West, Rathnelly, South Hill, Forest..."
6,Central Toronto,M5N,43.711695,-79.416936,Roselawn
7,Central Toronto,M5P,43.696948,-79.411307,"Forest Hill North & West, Forest Hill Road Park"
8,Central Toronto,M5R,43.672710,-79.405678,"The Annex, North Midtown, Yorkville"
9,Downtown Toronto,M4W,43.679563,-79.377529,Rosedale


Merge with the Lat/Long of Toronto, ON

In [167]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


<B>Filter the DF to only work with "Toronto area" Boroughs. </b>

In [168]:
all_toronto_data = tor_neigh[tor_neigh['Borough'].str.contains('Toronto', na = True)] # 
all_toronto_data.head(44)

,Borough,Postalcode,Latitude,Longitude,Neighborhood
0,Central Toronto,M4N,43.728020,-79.388790,Lawrence Park
1,Central Toronto,M4P,43.712751,-79.390197,Davisville North
2,Central Toronto,M4R,43.715383,-79.405678,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,43.704324,-79.388790,Davisville
4,Central Toronto,M4T,43.689574,-79.383160,"Moore Park, Summerhill East"
5,Central Toronto,M4V,43.686412,-79.400049,"Summerhill West, Rathnelly, South Hill, Forest..."
6,Central Toronto,M5N,43.711695,-79.416936,Roselawn
7,Central Toronto,M5P,43.696948,-79.411307,"Forest Hill North & West, Forest Hill Road Park"
8,Central Toronto,M5R,43.672710,-79.405678,"The Annex, North Midtown, Yorkville"
9,Downtown Toronto,M4W,43.679563,-79.377529,Rosedale


In [169]:
print('The "Contains Toronto" dataframe has {} boroughs and {} neighborhoods.'.format(
        len(all_toronto_data['Borough'].unique()),
        all_toronto_data.shape[0]
    )
)

The "Contains Toronto" dataframe has 4 boroughs and 39 neighborhoods.


In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
    
for lat, lng, borough, neighborhood in zip(tor_neigh['Latitude'], tor_neigh['Longitude'], tor_neigh['Borough'], tor_neigh['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#990000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 2c - Find the Neighborhood with the Airport

The goal is to Create a map of the nearest Pub-type places to the Airport so my group can fly into Toronto and do a pub crawl near the airport and be able to walk the whole way.

Find the Neighborhood that has the airport and then find nearest with the most pubs

In [170]:
target_loc = tor_neigh[tor_neigh['Neighborhood'].str.contains('University', na = True)] # 
target_loc 

,Borough,Postalcode,Latitude,Longitude,Neighborhood
21,Downtown Toronto,M5S,43.662696,-79.400049,"University of Toronto, Harbord"
63,North York,M3J,43.767980,-79.487262,"Northwood Park, York University"


The Univversity of Toronto is in Loc=21, so we'll start there to find the areas with the most pubs

In [171]:

neighborhood_latitude = tor_neigh.loc[21, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tor_neigh.loc[21, 'Longitude'] # neighborhood longitude value

neighborhood_name = tor_neigh.loc[21, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of University of Toronto, Harbord are 43.6626956, -79.4000493.


## Sect 3 - Import Business Directory data from FourSquare

In [172]:
CLIENT_ID = 'KHKXZNF5X32XKEAPPSZZNQ3EGJU2FGAIKE00LUQBMKNP4HGG' # your Foursquare ID
CLIENT_SECRET = 'IXUDUIXQH1QHNPWTPL4YDCVEWNJVBGJ1DD0GGFRBHMBNUJDA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KHKXZNF5X32XKEAPPSZZNQ3EGJU2FGAIKE00LUQBMKNP4HGG
CLIENT_SECRET:IXUDUIXQH1QHNPWTPL4YDCVEWNJVBGJ1DD0GGFRBHMBNUJDA


Search a 800 meter (about 1/2 mile) area around the Airport for "pubs"

In [173]:
latitude = neighborhood_latitude
longitude = neighborhood_longitude

search_query = 'restaurants'
radius = 800
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
#url


'https://api.foursquare.com/v2/venues/explore?client_id=KHKXZNF5X32XKEAPPSZZNQ3EGJU2FGAIKE00LUQBMKNP4HGG&client_secret=IXUDUIXQH1QHNPWTPL4YDCVEWNJVBGJ1DD0GGFRBHMBNUJDA&ll=43.6626956,-79.4000493&v=20180605&radius=800&limit=100'

In [174]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f151baa94c11e5e3be50ee1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'University of Toronto',
  'headerFullLocation': 'University of Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 131,
  'suggestedBounds': {'ne': {'lat': 43.669895607200004,
    'lng': -79.3901151046533},
   'sw': {'lat': 43.655495592799994, 'lng': -79.40998349534671}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5362c366498e602fbe1db395',
       'name': 'Yasu',
       'location': {'address': '81 Harbord St.',
        'lat': 43.66283719650635,
        'lng': -79.40321739973975,
        'labeledLatLngs': [{'label': 'display',
        

In [175]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



My end result was that I found 13 businesses within the 800 meters of the Airport.

In [176]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(125)

,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Rasa,Restaurant,43.662757,-79.403988
2,Her Father's Cider Bar + Kitchen,Beer Bar,43.662448,-79.404703
3,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746
4,Almond Butterfly,Bakery,43.662836,-79.403365
5,Piano Piano,Italian Restaurant,43.662949,-79.402898
6,Harbord House,Bar,43.662466,-79.405410
7,Akai Sushi,Sushi Restaurant,43.662470,-79.404946
8,Harvest Kitchen,Restaurant,43.662714,-79.404330
9,Sivananda Yoga Centre,Yoga Studio,43.662754,-79.402951


Since I did not have enough "Pubs", I expanded "Pubs" to include "BBQ Joint", "Pizza Place", "Burger Joint", and "Bar".

In [ ]:
#pub_places = nearby_venues[nearby_venues['categories'] == 'BBQ Joint'].reset_index(drop=True)
#pub_places = nearby_venues[nearby_venues['categories'].str.contains('bbq', na = True)] # 
#toronto_data = tor_neigh[tor_neigh['Borough'].str.contains('Toronto', na = True)] # 
pub_places = nearby_venues[nearby_venues['categories'].str.contains('Coffee|Cafe|Donut', na = True)] # 
pub_places.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
print('{} coffee places were near to the University.'.format(pub_places.shape[0]))

In [ ]:
latitude = neighborhood_latitude
longitude = neighborhood_longitude

# create map of Toronto using latitude and longitude values
map_tor_pubs = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
# for lat, lng, borough, neighborhood in zip(pub_places['Latitude'], pub_places['Longitude'], pub_places['Borough'], pub_places['Neighborhood']):
for lat, lng, categories, name in zip(pub_places['lat'], pub_places['lng'], pub_places['name'], pub_places['categories']):
    label = '{}, {}'.format(categories,name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.9,
        parse_html=False).add_to(map_tor_pubs)  
    
map_tor_pubs

### Cluster by neighborhoods
Week 4 Project starts here

In [177]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [178]:
toronto_venues = getNearbyVenues(names=all_toronto_data['Neighborhood'],
                                   latitudes=all_toronto_data['Latitude'],
                                   longitudes=all_toronto_data['Longitude']
                                  )

#toronto_venues = getNearbyVenues(names=tor_neigh['Neighborhood'],
#                                   latitudes=tor_neigh['Latitude'],
#                                   longitudes=tor_neigh['Longitude']
#                                  )



Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government
The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Business reply mail Processing Centre, South Central Letter

In [179]:
print(toronto_venues.shape)
toronto_venues.head()

(1636, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.72802,-79.38879,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.72802,-79.38879,The Photo School – Toronto,43.730429,-79.388767,Photography Studio
2,Lawrence Park,43.72802,-79.38879,cheltenham park,43.726670,-79.385324,Park
3,Lawrence Park,43.72802,-79.38879,Zodiac Swim School,43.728532,-79.382860,Swim School
4,Lawrence Park,43.72802,-79.38879,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line


In [180]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,66,66,66,66,66,66
Christie,16,16,16,16,16,16
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [181]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


## Sect 4 - Neighborhood Analysis - <br> Compare venue category counts in each Neighborhood for ALL Venue Categories

In [182]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [183]:
toronto_onehot.shape

(1636, 229)

In [184]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.071429,0.142857,0.214286,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.015152,0.000000,0.000000,0.015152,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.027397,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,...,0.013699,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.029412,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [185]:
toronto_grouped.shape

(39, 229)

In [186]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2      Farmers Market  0.03
3                Café  0.03
4  Seafood Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.05
4   Convenience Store  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
           venue  freq
0    Yoga Studio  0.07
1  Garden Center  0.07
2     Comic Shop  0.07
3    Pizza Place  0.07
4     Restaurant  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.21
1      Airport Lounge  0.14
2    Airport Terminal  0.14
3     Harbor / Marina  0.07
4  Airport Food Court  0.07


----Central Bay Street----
                 venue  freq
0          Co

In [187]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [188]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(85)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Café,Beer Bar,Cheese Shop,Pharmacy,Seafood Restaurant,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Burrito Place,Italian Restaurant,Intersection,Stadium,Restaurant,Bar
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Pizza Place,Brewery,Light Rail Station,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Bubble Tea Shop,Salad Place,Burger Joint,Bar,Thai Restaurant
5,Christie,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Candy Store,Restaurant,Baby Store,Coffee Shop
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Hotel,Yoga Studio,Men's Store,Mediterranean Restaurant,Pub
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Deli / Bodega,Japanese Restaurant,Seafood Restaurant,Italian Restaurant
8,Davisville,Dessert Shop,Sandwich Place,Café,Pizza Place,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pharmacy,Brewery
9,Davisville North,Hotel,Gym / Fitness Center,Breakfast Spot,Food & Drink Shop,Sandwich Place,Pizza Place,Department Store,Gym,Park,Doner Restaurant


### Cluster the Neighborhoods - Short list of Venue Categories

In [189]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 4, 0, 0, 0, 0, 0, 0], dtype=int32)

In [190]:
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.071429,0.142857,0.214286,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.015152,0.000000,0.000000,0.015152,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.027397,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,...,0.013699,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.029412,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [191]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = all_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Postalcode,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,M4N,43.728020,-79.388790,Lawrence Park,2,Park,Photography Studio,Bus Line,Swim School,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,Central Toronto,M4P,43.712751,-79.390197,Davisville North,0,Hotel,Gym / Fitness Center,Breakfast Spot,Food & Drink Shop,Sandwich Place,Pizza Place,Department Store,Gym,Park,Doner Restaurant
2,Central Toronto,M4R,43.715383,-79.405678,"North Toronto West, Lawrence Park",0,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Ice Cream Shop,Rental Car Location,Café,Mexican Restaurant,Fast Food Restaurant,Salon / Barbershop
3,Central Toronto,M4S,43.704324,-79.388790,Davisville,0,Dessert Shop,Sandwich Place,Café,Pizza Place,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pharmacy,Brewery
4,Central Toronto,M4T,43.689574,-79.383160,"Moore Park, Summerhill East",3,Park,Gym,Tennis Court,Trail,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant


Visualize Clusters

In [192]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In [193]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4P,0,Hotel,Gym / Fitness Center,Breakfast Spot,Food & Drink Shop,Sandwich Place,Pizza Place,Department Store,Gym,Park,Doner Restaurant
2,M4R,0,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Ice Cream Shop,Rental Car Location,Café,Mexican Restaurant,Fast Food Restaurant,Salon / Barbershop
3,M4S,0,Dessert Shop,Sandwich Place,Café,Pizza Place,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pharmacy,Brewery
5,M4V,0,Pub,Coffee Shop,Bagel Shop,Bank,Supermarket,Sushi Restaurant,Sports Bar,Restaurant,Fried Chicken Joint,Pizza Place
8,M5R,0,Sandwich Place,Café,Coffee Shop,Park,History Museum,Burger Joint,Liquor Store,Martial Arts Dojo,Indian Restaurant,Middle Eastern Restaurant
10,M4X,0,Coffee Shop,Pizza Place,Café,Chinese Restaurant,Bakery,Park,Italian Restaurant,Pub,Convenience Store,Restaurant
11,M4Y,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Hotel,Yoga Studio,Men's Store,Mediterranean Restaurant,Pub
12,M5A,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Dessert Shop,Shoe Store,Restaurant
13,M5B,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Hotel,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Bookstore,Electronics Store
14,M5C,0,Restaurant,Café,Coffee Shop,Clothing Store,Cocktail Bar,Cosmetics Shop,American Restaurant,Italian Restaurant,Seafood Restaurant,Beer Bar


In [194]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M5N,1,Health & Beauty Service,Home Service,Garden,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [195]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,2,Park,Photography Studio,Bus Line,Swim School,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [196]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4T,3,Park,Gym,Tennis Court,Trail,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant
7,M5P,3,Park,Jewelry Store,Trail,Sushi Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,M4W,3,Park,Playground,Trail,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [197]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M5V,4,Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina,Boat or Ferry


# Sect 5 - Neighborhood Analysis - <BR> Compare venue category counts in each Neighborhood for TARGETED Venue Categories

In [198]:
toronto_grouped_tar = toronto_grouped[['Neighborhood','Yoga Studio','Gym','Gym / Fitness Center','Hotel','Convention Center','Park','Trail','Garden','Baseball Stadium','Basketball Stadium','College Auditorium','College Rec Center','Event Space','General Entertainment','Performing Arts Venue','College Gym','Martial Arts Dojo','Skate Park','Skating Rink','Tennis Court','Video Game Store','Spa']]
toronto_grouped_tar

,Neighborhood,Yoga Studio,Gym,Gym / Fitness Center,Hotel,Convention Center,Park,Trail,Garden,Baseball Stadium,...,Event Space,General Entertainment,Performing Arts Venue,College Gym,Martial Arts Dojo,Skate Park,Skating Rink,Tennis Court,Video Game Store,Spa
0,Berczy Park,0.000000,0.017241,0.000000,0.017241,0.00,0.017241,0.00000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.045455,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00,...,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.066667,0.000000,0.066667,0.000000,0.00,0.066667,0.00000,0.066667,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,0.066667
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,Central Bay Street,0.015152,0.000000,0.015152,0.015152,0.00,0.015152,0.00000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.015152
5,Christie,0.000000,0.000000,0.000000,0.000000,0.00,0.125000,0.00000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,Church and Wellesley,0.027397,0.013699,0.000000,0.027397,0.00,0.013699,0.00000,0.000000,0.00,...,0.000000,0.013699,0.000000,0.000000,0.013699,0.000000,0.00,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.040000,0.010000,0.060000,0.00,0.010000,0.00000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.058824,0.000000,0.000000,0.00,0.029412,0.00000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.029412,0.000000,0.000000
9,Davisville North,0.000000,0.111111,0.111111,0.111111,0.00,0.111111,0.00000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [199]:
num_top_venues = 5

for hood in toronto_grouped_tar['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped_tar[toronto_grouped_tar['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0               Hotel  0.02
1                Park  0.02
2  Basketball Stadium  0.02
3                 Gym  0.02
4         Yoga Studio  0.00


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                    Gym  0.05
1  Performing Arts Venue  0.05
2            Yoga Studio  0.00
3       Video Game Store  0.00
4           Tennis Court  0.00


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
         venue  freq
0  Yoga Studio  0.07
1         Park  0.07
2   Skate Park  0.07
3       Garden  0.07
4          Spa  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0       Yoga Studio   0.0
1               Gym   0.0
2  Video Game Store   0.0
3      Tennis Court   0.0
4      Skating Rink   0.0


----Central Bay Street----
                  venue  freq
0           Yoga S

In [200]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [201]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped_tar['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped_tar.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(40)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Gym,Hotel,Park,Basketball Stadium,Spa
1,"Brockton, Parkdale Village, Exhibition Place",Gym,Performing Arts Venue,Spa,Basketball Stadium,Gym / Fitness Center
2,"Business reply mail Processing Centre, South C...",Spa,Garden,Gym / Fitness Center,Park,Yoga Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Spa,Video Game Store,Gym,Gym / Fitness Center,Hotel
4,Central Bay Street,Spa,Park,Gym / Fitness Center,Hotel,Yoga Studio
5,Christie,Park,Spa,Video Game Store,Gym,Gym / Fitness Center
6,Church and Wellesley,Yoga Studio,Hotel,Gym,Martial Arts Dojo,Park
7,"Commerce Court, Victoria Hotel",Hotel,Gym,Gym / Fitness Center,Park,Spa
8,Davisville,Gym,Tennis Court,Park,Spa,Basketball Stadium
9,Davisville North,Gym,Gym / Fitness Center,Hotel,Park,Spa


#### Cluster Neighborhood

In [202]:
# set number of clusters
kclusters = 10

toronto_grouped_tar_clustering = toronto_grouped_tar.drop('Neighborhood', 1)
toronto_grouped_clustering
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_tar_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 0, 9, 0, 0, 8, 0, 0, 0, 6, 8, 0, 4, 0, 0, 0, 8, 0, 3, 0],
      dtype=int32)

In [203]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = all_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Postalcode,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,M4N,43.728020,-79.388790,Lawrence Park,3,Park,Spa,Video Game Store,Gym,Gym / Fitness Center
1,Central Toronto,M4P,43.712751,-79.390197,Davisville North,6,Gym,Gym / Fitness Center,Hotel,Park,Spa
2,Central Toronto,M4R,43.715383,-79.405678,"North Toronto West, Lawrence Park",9,Spa,Park,Gym / Fitness Center,Yoga Studio,Skate Park
3,Central Toronto,M4S,43.704324,-79.388790,Davisville,0,Gym,Tennis Court,Park,Spa,Basketball Stadium
4,Central Toronto,M4T,43.689574,-79.383160,"Moore Park, Summerhill East",1,Tennis Court,Gym,Park,Trail,Spa


In [1]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'folium' is not defined

In [205]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,M4S,0,Gym,Tennis Court,Park,Spa,Basketball Stadium
5,M4V,0,Spa,Video Game Store,Gym,Gym / Fitness Center,Hotel
8,M5R,0,Martial Arts Dojo,Park,Spa,Basketball Stadium,Gym
10,M4X,0,Park,Gym / Fitness Center,General Entertainment,Spa,Basketball Stadium
11,M4Y,0,Yoga Studio,Hotel,Gym,Martial Arts Dojo,Park
13,M5B,0,Hotel,Spa,Gym,Gym / Fitness Center,Park
14,M5C,0,Gym,Hotel,Park,Performing Arts Venue,Spa
15,M5E,0,Gym,Hotel,Park,Basketball Stadium,Spa
16,M5G,0,Spa,Park,Gym / Fitness Center,Hotel,Yoga Studio
17,M5H,0,Gym,Hotel,Gym / Fitness Center,Event Space,Spa


In [206]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,M4T,1,Tennis Court,Gym,Park,Trail,Spa


In [207]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,M4W,2,Park,Trail,Spa,Video Game Store,Gym


In [208]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4N,3,Park,Spa,Video Game Store,Gym,Gym / Fitness Center


In [209]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,M5P,4,Park,Trail,Spa,Video Game Store,Gym


In [210]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,M5N,5,Garden,Spa,Video Game Store,Gym,Gym / Fitness Center


In [211]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,M4P,6,Gym,Gym / Fitness Center,Hotel,Park,Spa


In [212]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
28,M4E,7,Trail,Spa,Video Game Store,Gym,Gym / Fitness Center


In [213]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,M5A,8,Park,Spa,Performing Arts Venue,Gym / Fitness Center,Event Space
26,M6G,8,Park,Spa,Video Game Store,Gym,Gym / Fitness Center
30,M4L,8,Park,Gym,Spa,Video Game Store,Gym / Fitness Center
92,M6H,8,Park,Spa,Video Game Store,Gym,Gym / Fitness Center


In [214]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,M4R,9,Spa,Park,Gym / Fitness Center,Yoga Studio,Skate Park
32,M7Y,9,Spa,Garden,Gym / Fitness Center,Park,Yoga Studio
